# 人脸识别
这个练习的目的是利用主成分分析（PCA）和线性判别分析（LDA）进行人脸识别的图像。在编码前先阅读所有指令。

## 1. 导入数据  
下载face.zip并从face.zip提取图像。你会发现一些人脸图像来自CMU PIE数据库。这里有10个人，每个人在24种不同的照明条件下拍摄，共有240幅图像。为了方便起见，这些图像在训练和测试目录中被分成两个相等的集合。  
图像的文件名ppppp_xx_yy.bmp，这里ppppp表示人的身份；xx表示头部方向（所有的都是正面的（xx = 27）在这个练习中）; YY表示照明条件。所有图像都被裁剪并对齐，它们的高度、宽度分别为160和140像素。

## 2. 特征提取  
特征提取在人脸识别中起着重要作用。好的特性应该能够区分不同的用户。在这部分中，您将学习如何提取PCA特征和LDA特征。  
 PCA 
这部分将指导您从图像的训练集学习PCA特征提取器。您还将学习如何用图像提取器提取PCA特征。  
在FisherFace.py学习的代码。使用这些函数的PCA特征提取：mypca，阅读面。他们的目的是在代码中描述的。遵照指示：  
1. 读取数据。通过将参数设置为训练文件夹的路径，使用read_faces读取所有训练图像。现在你应该有一个$22400\times120$的脸矩阵，它的列是训练目录中所有的人脸图像，以及相应的标签范围从0到9。  
2. 训练PCA。使用mypca函数，计算PCA投影矩阵$W$，全局均值向量$m_e$和特征值向量。学习代码- mypca，理解这是如何实现的。特别是，注意内积技巧的使用避免出现“内存不足”错误。    
3. 选择的特征维度。只保留前$K(K=30)$特征脸，通过公式$ W_e = W[:,:K]$  
4. 投影图像。对于一个人脸图像x，通过公式$y = W_e^T(x-m_e)$将它投影到PCA空间中。y是x的PCA特征表示。 
5. 反向投影PCA特征。众所周知，PCA也经常被用作降维方法。所以我们可以在已学习的PCA空间由特征向量y重建图像x：$x = W_ey+m_e$。

In [ ]:
def myPCA(A):
    # function [W,LL,m]=mypca(A)
    # computes PCA of matrix A
    # A: D by N data matrix. Each column is a random vector
    # W: D by K matrix whose columns are the principal components in
    # decreasing order
    # LL: eigenvalues
    # m: mean of columns of A
    
    # Note: 'lambda' is a Python reserved word
    
    # compute mean, and substract mean from every column
    [r,c] = A.shape
    m = np.mean(A,1)
    A = A - np.tile(m,(c,1)).T
    B = np.dot(A.T, A)
    [d, v] = linalg.eig(B)
    
    # sort d in descending order
    order_index = np.argsort(d)
    order_index = order_index[::-1]
    d = d[order_index]
    v = v[:,order_index]
    
    # compute eigenvectors of scatter matrix
    W = np.dot(A,v)
    Wnorm = ComputeNorm(W)
    
    W1 = np.tile(Wnorm, (r,1))
    W2 = W/W1
    
    LL = d[0:-1]
    
    W = W2[:,0:1]
    
    return W,LL,m 

In [3]:
import os
import numpy as np
import scipy.linalg as linalg
import cv2
import operator

def ComputerNorm(x):
    # function r=ComputerNorm(x)
    # computes vector norms of x
    # x : d x m matrix, each column a vector
    # r : 1 x m matrix, each the corresponding norm (L2)
    [row, col] = x.shape
    r = np.zeros((1,col))
    
    for i in range(col):
        r[0,i] = linalg.norm(x[:,i])
    return r  

def myLDA(A, Labels):
    # function [W,m] = myLDA(A, Label)
    # computes LDA of matrix A
    # A: D by N data matrix. Each column is a random vector. 
    # W: D by K matrix whose columns are the principal components in
    # decreasing order
    # m: mean of each projection
    classLabels = np.unique(Labels)
    classNum = len(classLabels)
    dim, datanum = A.shape
    totalMean = np.mean(A,1)
    partition = [np.where(Labels == label)[0] for label in classLabels]
    classMean = [(np.mean(A[:,idx],1),len(idx)) for idx in partition]
    
    # compute the within-class scatter matrix
    W = np.zeros((dim,dim))
    for idx in partition:
        W += np.cov(A[:,idx],rowvar = 1)*len(idx)
        
    # compute the between-class scatter matrix
    B = np.zeros((dim,dim))
    for mu, class_size in classMean:
        offset = mu - totalMean
        B += np.outer(offset, offset)*class_size
    
    
def read_faces(directory):
    # function faces = read_faces(directory)
    # Browse the directory, read image files and store faces in a matrix
    # faces: face matrix in which each column is a column vector for 1 face image
    # idLabels: corresponding ids for face matrix
    
    A = [] # A will store list of image vectors
    Label = [] # Label will store list of identity label
    

In [27]:
Labels = np.array([1,1,2,2,4,4,3,2,1,4])

In [29]:
classLabels = np.unique(Labels)
classLabels

array([1, 2, 3, 4])

In [31]:
classNum = len(classLabels)
classNum 

4